In [ ]:
X = df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]]
y = df.iloc[:,2:3]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
catb_model = CatBoostRegressor().fit(X_train, y_train)

y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

catb_params = {"iterations": [200,500,100,999],
              "learning_rate": [0.01,0.1,0.05],
              "depth": [1,3,6]}

catb_model = CatBoostRegressor()

catb_cv_model = GridSearchCV(catb_model, 
                           catb_params, 
                           cv = 5,  
                           verbose = 2).fit(X_train, y_train)

catb_cv_model.best_params_

catb_tuned = CatBoostRegressor(depth = 1, iterations = 100, learning_rate = 0.9).fit(X_train, y_train)

ypred=catb_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, y_pred)
print('r2 = {}'.format(model))

df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)
# Predicting for whole dataset
df['y_pred'] =  catb_tuned.predict(df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]])

# Create Interactive Plot Using Bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
output_notebook() 

p = figure(plot_width=800, 
           plot_height=400,
           title="PV Electricity Consumption Prediction",
           x_axis_label='Date and Time',
           y_axis_label='Electricity Consumption',
           x_axis_type="datetime")

p.line(df.index, df['Load'], legend='Observed Electricity Consumption', line_width=1)
p.line(df.index, df['y_pred'], legend='Predicted Electricity Consumption', line_width=1, line_color="red")

show(p, notebook_handle = True)

Importance = pd.DataFrame({'Importance':catb_tuned.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None